# Exercise 4 Camera calibration 

In this exercise we will look at:
- Direct linear transform (DLT), linear algorithm for camera calibration and
- Checkerboard calibration, and bundle adjustment from Zhang (2000).

You should be able to perform camera calibration using both methods.

In [248]:
from ex1 import box3d, projectpoints
import matplotlib.pyplot as plt
import math
import numpy as np
import cv2
from scipy.spatial.transform import Rotation

4.1 DLT

In [249]:
# Pose matrices:
R = np.array([[np.sqrt(0.5), -np.sqrt(0.5), 0], [np.sqrt(0.5), np.sqrt(0.5), 0], [0, 0, 1]])
t = np.array([[0, 0, 10]]).T

f = 1000
dx = 1920//2
dy = 1080//2
K = np.array([[f, 0, dx], [0, f, dy], [0, 0, 1]])
K

array([[1000,    0,  960],
       [   0, 1000,  540],
       [   0,    0,    1]])

In [250]:
# Exact 3D points
Q = np.array([[0, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 0, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1]])
Q

array([[0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1]])

In [252]:
Pro = K@np.hstack((R, t))
Pro.astype(int)

array([[ 707, -707,  960, 9600],
       [ 707,  707,  540, 5400],
       [   0,    0,    1,   10]])

In [253]:
Q_h = np.vstack((Q, np.ones(len(Q[0]))))

In [254]:
q_h = Pro@Q_h
q = q_h[0:2, :]/q_h[2, :]
for i in range(len(q[0])):
    print(np.round(q[:,i],2), end=", ")

[960. 540.], [1030.71  610.71], [889.29 610.71], [960.   681.42], [960. 540.], [1024.28  604.28], [895.72 604.28], [960.   668.56], 

#### 4.2 DLT: Use Q and q only to estimate P

In [255]:
def crossop(q):
    return np.array([[0, -q[2], q[1]], [q[2], 0, -q[0]], [-q[1], q[0], 0]])
    
def get_b(Q,q):
    """Calculates the B matrix by using kronecker multiplication and crossup."""
    B = np.kron(Q[:,0], crossop(q[:,0]))
    for i in range(1,len(q[0])):
        B_temp = np.kron(Q[:,i], crossop(q[:,i]))
        B = np.vstack((B,B_temp))
    return B

B = get_b(Q_h, q_h)

In [256]:
_,_,VT = np.linalg.svd(B.T@B) #U,S,VT = np.linalg.svd(A.T@A)
# VT is sorted in descending order of singular value
P =  VT[-1] # VT is flattened, P is a 3x4
P = P.reshape((4,3)).T

In [257]:
np.int32((P/P[-1, -1] * 10)) # Getting it on the same format as the original P

array([[ 707, -707,  960, 9599],
       [ 707,  707,  540, 5399],
       [   0,    0,    1,   10]])

In [258]:
Pro.astype(int)

array([[ 707, -707,  960, 9600],
       [ 707,  707,  540, 5400],
       [   0,    0,    1,   10]])

In [259]:
## Reproject the points Q: q_h = P@Q
q_h_est = P@Q_h
q_est = q_h_est[0:2, :]/q_h_est[2, :]

In [260]:
for i in range(len(q[0])):
    print(np.round(q_est[:,i],2), end="\t")

[960. 540.]	[1030.71  610.71]	[889.29 610.71]	[960.   681.42]	[960. 540.]	[1024.28  604.28]	[895.72 604.28]	[960.   668.56]	

In [261]:
np.linalg.norm(q_est-q)

1.062121817141783e-08

#### Try normalizing before using SVD

In [262]:
## What about normalizing points before using SVD?
def normalize2d(Q):
    mean = np.mean(Q,axis=1)
    std = np.std(Q,axis=1)
    T = np.array([[1/std[0],0,-mean[0]/std[0]],
                  [0,1/std[1],-mean[1]/std[1]],
                 [0,0,1]])
    return [T@Q, T]

Normalizing Q  before doing SVD for a more stable algorighm.

In [263]:
normed_points,T = normalize2d(q_h)
B = get_b(Q_h, normed_points)
_,_,VT = np.linalg.svd(B.T@B) #U,S,VT = np.linalg.svd(A.T@A)
# VT is sorted in descending order of singular value
P_norm =  VT[-1]
P_norm = P_norm.reshape((4,3)).T # VT is flattened and transposed, P is a 3x4
P_norm = np.linalg.inv(T)@P_norm # Turn back the normalization

In [264]:
q_h_norm_est = P_norm@Q_h
q_norm_est = q_h_norm_est[0:2, :]/q_h_norm_est[2, :]

In [271]:
print("Estimatet P, normalized\n",np.int32(np.round((P_norm/P_norm[-1, -1] * 10),2)))
print("Estimated P, not normalized\n",np.int32(np.round((P/P[-1, -1] * 10),2)))
print("Original P\n",Pro.astype(int))

Estimatet P, normalized
 [[ 707 -707  960 9600]
 [ 707  707  540 5400]
 [   0    0    1   10]]
Estimated P, not normalized
 [[ 707 -707  960 9600]
 [ 707  707  540 5400]
 [   0    0    1   10]]
Original P
 [[ 707 -707  960 9600]
 [ 707  707  540 5400]
 [   0    0    1   10]]


In [272]:
print(np.linalg.norm(q_est-q))
print(np.linalg.norm(q_norm_est-q))

1.062121817141783e-08
3.822212751818915e-08


## Programming: Checkerboard calibration using Zhangs method

Here we will prepare calibration with checkerboards. We do not yet have the ability to detect
checkerboards, so for now we will define the points ourselves.

#### 4.3 Define a checkerboard


In [155]:
def checkerboard_points(n,m):
    Q = np.zeros(shape=(3, m*n))
    for i in range(n):
        for j in range(m):
            Q[:,i+j] = [i - (n-1)/2,
                        j - (m-1)/2,
                        0]
                
    return Q

In [161]:
chessboard = checkerboard_points(8,8)
chessboard.shape

(3, 64)

#### 4.4 Checkerboard corners

```
R = Rotation.from_euler('xyz', [θx, θy, θz]).as_matrix()
```

In [ ]:
from scipy.spatial.transform import Rotation
